<a href="https://github.com/xtHuang0927/GTFS2GMNS" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
- **GTFS2GMNS**: The General Transit Feed Specification (GTFS) defines a common format for public transportation schedules and associated geographic information. It is used by thousands of public transport providers. As a data conversion tool, gtfs2gmns, can directly convert the GTFS data to node, link, and agent files in the GMNS format. In addition, this tool can merge the transit network into the road network which is obtain by Open Street Map via OSM2GMNS.

- **AMS**: The objective of the AMS data hub is to define a prototype of operations and associated requirements that will allow for the effective integration of analysis modeling and simulation tools across various domains and scales. Easily exchange information and data at both the input and output levels.

- **ASU Trans + AI Lab**: The team devotes to developing a better AMS data hub framework and a set of integrated open-source transportation tools. The objective of this notebook is to aggregate the developed software as a systematic modeling flow and demonstrate its successive steps, so call “7-steps modeling”. 


In [ ]:
!pip install gtfs2gmns
!pip install osm2gmns
!pip install geopandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
"import packages"
import os

### Step 0. GTFS data download

https://gtfs.org/examples/

https://transitfeeds.com/

| Step |     Description    |       Software       | Input Files | Output Files |
|:----:|:------------------:|:--------------------:|:-----------:|:------------:|
    | 0    | GTFS data download | transitfeeds | - -         | stop.txt, route.txt, trip.txt, stop_times.txt and others|

### Step 0.1 Convert GTFS Data into GMNS Format

In [1]:
"Step 0.1 Convert GTFS Data into GMNS Format"

gtfs_path = '.'
gmns_path = '.'

import gtfs2gmns as gg

node_transit,link_transit = gg.Convert_GTFS(gtfs_path,gmns_path)

# Please modify the directory.
# If you need the bounding box of the transit network, Create_Boundary function might help.

import gtfs2gmns as gg

node_transit,link_transit = gg.Convert_GTFS(gtfs_path,gmns_path)
bbox = gg.Create_Boundary(node_transit)

gtfs2gmns, version 0.1.0
reading gtfs data...
converting gtfs data into gmns format...
reading gtfs data...
converting gtfs data into gmns format...


In [3]:
node_transit.head(5)

,node_id,stop_id,x_coord,y_coord,node_type,ctrl_type,zone_id,geometry
0,10000001,2320943,-78.616821,35.768137,transit,NaN,NaN,POINT (-78.61682 35.76814)
1,10000002,2323762,-78.693720,35.793506,transit,NaN,NaN,POINT (-78.69372 35.79351)
2,10000003,2326845,-78.664027,35.789222,transit,NaN,NaN,POINT (-78.66403 35.78922)
3,10000004,2330752,-78.608443,35.797145,transit,NaN,NaN,POINT (-78.60844 35.79715)
4,10000005,2387286,-78.709050,35.793469,transit,NaN,NaN,POINT (-78.70905 35.79347)


In [4]:
link_transit.head(5)

,name,from_node_id,to_node_id,length,geometry,link_type_name,link_type,dir_flag,lanes,free_speed,capacity
link_id,,,,,,,,,,,
10000001,1537,10000839,10000840,196.408590,LINESTRING (-78.659554834459 35.76157155872600...,transit,99,1,1,50,100
10000002,1537,10000840,10000841,295.596966,"LINESTRING (-78.658098 35.762884, -78.655556 3...",transit,99,1,1,50,100
10000003,1537,10000841,10001027,237.175552,"LINESTRING (-78.655556 35.764561, -78.653533 3...",transit,99,1,1,50,100
10000004,1537,10001027,10000842,168.818786,"LINESTRING (-78.653533 35.765923, -78.652106 3...",transit,99,1,1,50,100
10000005,1537,10000842,10000569,601.512970,"LINESTRING (-78.652106 35.766905, -78.647961 3...",transit,99,1,1,50,100


In [5]:
bbox

[-78.80199, 35.573104, -78.323486, 35.990455]

### Step 0.2 Get the OSM Network
Before merging the transit network into the road network, you need to download the osm data and convert it into the GMNS format first. OSM2GMNS python package will be a good choice.

In [7]:
"Step 0.2 Get the OSM Network"

import os
import osm2gmns as og

osm_path = 'osm/consolidated/'

net = og.getNetFromOSMFile('osm/map.osm',network_type=('auto'), default_lanes=True, default_speed=True)
og.outputNetToCSV(net, output_folder='osm/')

net = og.getNetFromCSV(osm_path)
og.consolidateComplexIntersections(net)
og.outputNetToCSV(net, output_folder=osm_path)

### Step 1. Create the Connector
You can merge the networks by building the connector between the transit node and the nearby OSM node.

Make sure you already obtain node_transit.csv and the osm files (node.csv and link.csv).

In [8]:
"Step 1 Create the Connector"

import gtfs2gmns as gg

node,link_osm_connector = gg.CreatConnector_osm_gtfs(osm_path,gmns_path)

creating connector between osm network and gtfs data...


In [9]:
link_osm_connector.tail(5)

,name,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses
link_id,,,,,,,,,,,,
151609,None,10001472,55429,1,25.883684,1,29.0,None,connector,20,"LINESTRING (-78.780543291419 35.9139798803417,...",NaN
151610,None,17017,10001473,1,198.038204,1,29.0,None,connector,20,"LINESTRING (-78.7561799 35.773025399999995, -7...",NaN
151611,None,10001473,17017,1,198.038204,1,29.0,None,connector,20,"LINESTRING (-78.75837 35.773146000000004, -78....",NaN
151612,None,18078,10001474,1,44.517918,1,29.0,None,connector,20,"LINESTRING (-78.632828 35.7682416, -78.632341 ...",NaN
151613,None,10001474,18078,1,44.517918,1,29.0,None,connector,20,"LINESTRING (-78.632341 35.768306, -78.632828 3...",NaN


### Step 2. Create the Transit Route
This procedure can help you generate the actual trace for the transit route.


In [10]:
"Step 2 Create the Transit Route"

import gtfs2gmns as gg

link = gg.Create_TransitRoute(gmns_path)

creating transit routes...


D:\Users\Alice\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
link.tail(5)

,link_id,name,from_node_id,to_node_id,dir_flag,length,lanes,free_speed,capacity,link_type_name,link_type,geometry,allowed_uses
153460,10075761,1528,10000239,10000657,1,707.328793,1,50.0,100.0,transit,99,"LINESTRING (-78.692196 35.76867100000001, -78....",NaN
153461,10076132,1528,10000633,10000634,1,1003.325427,1,50.0,100.0,transit,99,"LINESTRING (-78.6762469992 35.7800731694, -78....",NaN
153462,10076133,1528,10000634,10000635,1,785.867076,1,50.0,100.0,transit,99,"LINESTRING (-78.681522538404 35.77511262308, -...",NaN
153463,10076136,1528,10001234,10000240,1,346.667968,1,50.0,100.0,transit,99,"LINESTRING (-78.6909845247 35.7693634746, -78....",NaN
153464,10076137,1528,10000240,10000915,1,1118.917643,1,50.0,100.0,transit,99,"LINESTRING (-78.694538 35.76835300000001, -78....",NaN


### Step 3. Visualization
You can visualize generated networks using NeXTA or QGIS.




---
| Step |         Description        |   Software  |                   Input Files                  |                       Output Files                       |
|:----:|:--------------------------:|:-----------:|:----------------------------------------------:|:--------------------------------------------------------:|
| 3    | Zone-to-zone travel demand | grid2demand | Node.csv, Link.csv, Poi.csv, Poi_trip_rate.csv | Demand.csv, Zone.csv, Accessibility.csv, Input_agent.csv |
